# **Frequency analysis in Python and MNE**

### **Importing one channel EEG in text format**

In [ ]:
# Importing necessary libraries
import numpy as np
import matplotlib.pyplot as plt

# Loading EEG data from a text file
data = np.loadtxt('One_Ch_EEG.txt')
data

In [ ]:
#Our one channel shape
print(data.shape)

In [ ]:
#Plotting one channel data
plt.plot(data)

### **Changing to second instead of points**

In [ ]:
# Defining the sampling rate
srate = 256

# Creating a time vector based on the length of the data and the sampling rate
time = np.arange(len(data)) / srate

plt.plot(time, data)
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.show()

### **Using FFT function for EEG data**

In [ ]:
# Calculating the power spectrum using Fast Fourier Transform (FFT)
powerSpectrum = np.fft.fft(data)

# Generating frequency values corresponding to the FFT result
freqs=np.fft.fftfreq(data.size, d=1/256)

In [ ]:
#Plot the result
fig, ax = plt.subplots(2,1)
ax[0].plot(time, data)
ax[0].set_xlabel('Time (s)')
ax[0].set_ylabel('Amplitude (uV)')

# Creating a mask to retain only positive frequencies
mask = freqs > 0
freqsPositive = freqs[mask]
spectrumPositive = np.abs(powerSpectrum)[mask]

ax[1].plot(freqsPositive, spectrumPositive)
ax[1].set_xlabel('Frequency (Hz)')
ax[1].set_ylabel('Amplitude (uV)')
ax[1].set_title('Amplitude Spectrum')
ax[1].set_xlim([1,40])


### **Frequency analysis in MNE**

In [ ]:
#Importing necessary libraries
import mne
from mne.preprocessing import ICA
import matplotlib.pyplot as plt
%matplotlib qt

#Importing the raw EEG data (EDF format) and importing into memory
EEG = mne.io.read_raw_fif('EEGFiltered.fif', preload=True)

#Filtering the data
EEG.filter(0.1,30)

#Create montage
montage = mne.channels.make_standard_montage('standard_1020')
#Set montage to raw EEG data
EEG.set_montage(montage)


### **Generate the Power Spectral Density (PSD)**

In [ ]:
psd, freqs =mne.time_frequency.psd_array_multitaper (EEG.get_data(), EEG.info['sfreq'],
                                                     fmin=0.5, fmax=30)
psd.shape

### **PSDs for 19 channels**

In [ ]:
# Creating a subplot to visualize power spectral density (PSD) for all EEG channels
fig, ax = plt.subplots(figsize=(20,10))
for i in range(len(EEG.ch_names)):
    ax.plot(freqs, psd[i], label=EEG.ch_names[i])
    ax.set_xlabel('Frequency Hz')
    ax.set_ylabel('PSDs for all 19 channels')
plt.legend()
plt.show()


In [ ]:
EEG.ch_names

### **PSD for desired channel**

In [ ]:
plt.plot(freqs, psd[18], label='O2')
plt.legend()

In [ ]:
# Printing the indices and names of all EEG channels
for i in range(len(EEG.ch_names)):
    print(i, EEG.ch_names[i])


In [ ]:
plt.plot(freqs, psd[8], label='C3')
plt.plot(freqs, psd[10], label='C4')
plt.legend()
plt.show()


### **Custom topomaps**

In [ ]:
im , _ = mne.viz.plot_topomap(psd.mean(axis=-1), pos=EEG.info, cmap='RdBu_r',
                              names=EEG.info['ch_names'])
plt.colorbar(im)

### **Topomaps for Theta, Alpha and Beta ranges**

In [ ]:
# Defining the frequency bands of interest
freqBands = [(3, 7), (8, 12), (13, 30)]

# Creating subplots for each frequency band
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
# Looping over each frequency band
for i, (fmin, fmax) in enumerate(freqBands):
    ax = axes[i]
     # Computing power spectral density (PSD) using multitaper method
    psd, freqs = mne.time_frequency.psd_array_multitaper(EEG.get_data(), sfreq=EEG.info['sfreq'], fmin=fmin, fmax=fmax)
    # Plotting topomap for the average PSD across channels
    mne.viz.plot_topomap(psd.mean(axis=-1), pos=EEG.info, cmap='RdBu_r', axes=ax, names=EEG.info['ch_names'])
    # Setting the title for the subplot
    ax.set(title='PSD ({:.1f} - {:.1f}) Hz'.format(fmin, fmax))
